Lenguaje de Definición de Datos
===

In [1]:
%load_ext bigdata
%hive_init

Hive initialized!


In [2]:
%%hive
-- prepara la db 
DROP DATABASE IF EXISTS DDLdb CASCADE;

---

## CREATE DATABASE

    CREATE (DATABASE|SCHEMA) [IF NOT EXISTS] database_name
       [COMMENT database_comment]
       [LOCATION hdfs_path]
       [WITH DBPROPERTIES (property_name=property_value, ...)];

In [3]:
%%hive
-- 
-- Crea la base de datos DDLdb
--
CREATE DATABASE DDLdb;

In [4]:
%%hive
--
-- crea la base de datos DDLdb únicamente si no existe
-- (note que aparece un warning)
--
CREATE DATABASE IF NOT EXISTS DDLdb COMMENT 'Database de prueba';

---
**Ejercicio.--** Describa el efecto del siguiente comando:
  
     CREATE DATABASE demo LOCATION ~/temp/directory ;
     
---

## SHOW DATABASES

In [5]:
%%hive
--
-- muestra las bases de datos existentes
-- 
SHOW DATABASES;

ddldb
default
sqldb


In [6]:
%%hive
SHOW DATABASES LIKE 'd*';

ddldb
default


## USE

In [7]:
%%hive
--
-- Se conecta a la base de datos llamada DDLdb
--
USE DDLdb;

---
**Ejercicio.--** Cuál es el resultado de los siguientes comandos:

    DESCRIBE DATABASE DDLdb;
    
    DESCRIBE DATABASE EXTENDED DDLdb;
    
---

## DROP DATABASE

    DROP (DATABASE|SCHEMA) [IF EXISTS] database_name [RESTRICT|CASCADE];
    
Por defecto se usa RESTRICT; esta opción genera un error si la base de datos no esta vacía. Si desea borrar también las tablas use la opción CASCADE.   

In [8]:
%%hive
--
-- Borra la base de datos
--
DROP DATABASE IF EXISTS DDLdb CASCADE;
SHOW DATABASES;

default
sqldb


---
**Ejercicio.--** Cuál es el significado de la opción CASCADE?.

**Ejercicio.--** Cree las bases de datos llamadas db1 y db2.

**Ejercicio.--** Conectese a la base de datos db1.

**Ejercicio.--** Conectese a la base de datos db2.

**Ejercicio.--** Borre las bases de datos db1 y db2.

---

Las bases de datos se componen de tablas relacionadas por uno o mas campos, lo que permite gestionar diferentes fuentes de información.

A continuación se presentan algunas funciones necesarias para la gestión de tablas:

## CREATE TABLE


* Forma 1:

      CREATE [TEMPORARY] [EXTERNAL] TABLE [IF NOT EXISTS] [db_name.]table_name    
        [(col_name data_type [COMMENT col_comment], ... [constraint_specification])]
        [COMMENT table_comment]
        [PARTITIONED BY (col_name data_type [COMMENT col_comment], ...)]
        [CLUSTERED BY (col_name, col_name, ...) [SORTED BY (col_name [ASC|DESC], ...)] 
          INTO num_buckets BUCKETS]
        [SKEWED BY (col_name, col_name, ...)
           ON ((col_value, col_value, ...), (col_value, col_value, ...), ...)
           [STORED AS DIRECTORIES]
        [
          [ROW FORMAT row_format] 
          [STORED AS file_format]
             | STORED BY 'storage.handler.class.name' [WITH SERDEPROPERTIES (...)]  
        ]
        [LOCATION hdfs_path]
        [TBLPROPERTIES (property_name=property_value, ...)]  
        [AS select_statement];
 
 
* Forma 2:

      CREATE [TEMPORARY] [EXTERNAL] TABLE [IF NOT EXISTS] [db_name.]table_name
      LIKE existing_table_or_view_name
      [LOCATION hdfs_path];
 
Detalles:  https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-Create/Drop/Alter/UseDatabase
 
 

In [9]:
%%hive
DROP DATABASE IF EXISTS DDLdb CASCADE;
CREATE DATABASE DDLdb;
USE DDLdb;

In [10]:
%%hive
--
-- crea la tabla
--
CREATE TABLE persons (
    id INT,
    firstname VARCHAR(10),
    surname VARCHAR(10),
    birthday TIMESTAMP,
    color VARCHAR(9),
    quantity INT
    );

---
**Ejercicio.--** Describa que devuelve el comando:

    SHOW TABLES;
    
**Ejercicio.--** Describa que devuelve el comando:

    SHOW CREATE TABLE persons;
    
**Ejercicio.--** Describa que devuelve el comando:

    DESCRIBE persons;
    
    
**Ejercicio.--** Cuáles son los siguientes tipos de datos? Existen en MySQL?

* `STRUCT`


* `MAP`


* `ARRAY`

**Ejercicio.--** De ejemplos de uso de los tipos de datos anteriores.


---

In [11]:
%%hive
CREATE TABLE persons1 (
    id INT,
    firstname VARCHAR(10),
    surname VARCHAR(10),
    birthday TIMESTAMP,
    color VARCHAR(9),
    quantity INT)
COMMENT 'Esta es mi primera tabla'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
STORED AS TEXTFILE;

---
**Ejercicio.--** Donde se almacenó la tabla anteior?


**Ejercicio.--** Cuál es el resultado del comando:

     SHOW TBLPROPERTIES persons1

---


## LIKE

`LIKE`  permite copiar la estructura de una tabla (pero no sus datos).

In [12]:
%%hive
CREATE TABLE IF NOT EXISTS DDLdb.persons2 LIKE persons;

In [13]:
%%hive
SHOW TABLES IN DDLdb;

persons
persons1
persons2


## ALTER TABLE

    ALTER TABLE table_name RENAME TO new_table_name;

    ALTER TABLE table_name ADD COLUMNS (col_name data_type [COMMENT col_comment], ...)

    ALTER TABLE table_name REPLACE COLUMNS (col_name data_type [COMMENT col_comment], ...)


In [14]:
%%hive
-- 
-- Agrega una nueva columna especificando su posición.
--
ALTER TABLE persons ADD COLUMNS (state VARCHAR(8));
DESCRIBE persons;

id                  	int                 	                    
firstname           	varchar(10)         	                    
surname             	varchar(10)         	                    
birthday            	timestamp           	                    
color               	varchar(9)          	                    
quantity            	int                 	                    
state               	varchar(8)          	                    


---
**Ejercicio.--** Cuál es el resultado de los siguientes comandos?

     DESCRIBE persons;
     
     DESCRIBE EXTENDED persons;
     
     DESCRIBE FORMATED persons;
     
---

In [15]:
%%hive
-- 
-- Modifica el tipo de campo de una columna.
--
ALTER TABLE 
    persons 
CHANGE 
    firstname firstname VARCHAR(12);
DESCRIBE persons;

id                  	int                 	                    
firstname           	varchar(12)         	                    
surname             	varchar(10)         	                    
birthday            	timestamp           	                    
color               	varchar(9)          	                    
quantity            	int                 	                    
state               	varchar(8)          	                    


In [16]:
%%hive
--
-- Borra columnas.
--
ALTER TABLE 
    persons 
REPLACE COLUMNS 
    (id INT, 
     firstname VARCHAR(10));
DESCRIBE persons;

id                  	int                 	                    
firstname           	varchar(10)         	                    


In [17]:
%%hive
--
-- Borrado de tablas:
--
DROP TABLE IF EXISTS persons;
DROP TABLE IF EXISTS persons1;
DROP TABLE IF EXISTS persons2;

## EXTERNAL TABLES

Las tablas creadas hasta el momento son completamente manejadas por `Hive` y almacenadas en el directorio especificado en la propiedad `hive.metastore.warehouse.dir` del archivo `hive-site.xml`. Sin embargo, estas tablas no son convenientes para compartir directamente información con otras aplicaciones, ya que `Hive` administra los datos. Para analizar info generada por otras aplicaciones resulta más conveniente usar tablas externas. A continuación se presentan dos ejemplos:

    CREATE EXTERNAL TABLE IF NOT EXISTS db.table1 
    LIKE db.table
    LOCATION '/path/to/data';
    
    CREATE EXTERNAL TABLE IF NOT EXISTS table1 (
       col1 STRING,
       col2 STRING)
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' 
    LOCATION '/path/to/data';

## TABLAS PARTICIONADAS

Permiten dividir físicamente los datos para agilizar procesos de búsqueda y gestión de información. Por ejemplo:

    CREATE TABLE IF NOT EXISTS table (
       col1 STRING,
       col2 STRING,
       col3 STRING)
    PARTITIONED BY (col4 STRING, col5 STRING);
    
---
**Ejercicio.--** Cómo cambia la estrucutura en disco para una tabla particionada?

---

Para ver las particiones:

    SHOW PARTITIONS table;
    
    SHOW PARTITIONS table PARTITION (col4='x');
    
    
Las particiones también pueden crearse después de haber creado la tabla.

    ALTER TABLE table ADD PARTITION(col5 = xxx, col6 = xxxx)
    LOCATION '/path/to/data';
    
La partición también puede lograrse al momento de cargar los datos:

    LOAD DATA INPATH 'path/to/data'
    INTO TABLE table
    PARTITION(col4 = val, col5 = val);
    
También es posible usar este tipo de tablas con fuentes externas:

    CREATE EXTERNAL TABLE IF NOT EXISTS table1 (
       col1 STRING,
       col2 STRING)
    PARTITIONED BY (col3 INT)
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' 
    LOCATION '/path/to/data';

---